# 라이브러리

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib notebook
#import seaborn as sns

from sklearn.impute import SimpleImputer
#from sklearn.preprocessing import Imputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import tree

from sklearn.model_selection import train_test_split

import graphviz

import time

# Load data
 - HN16_ALL.sas7bdat : 설문지 데이터

In [3]:
df_data = pd.read_sas("./HN16_ALL.sas7bdat", format = 'sas7bdat', encoding='iso-8859-1')
print(df_data.shape)
df_data.head()

(8150, 768)


,mod_d,ID,ID_fam,year,region,town_t,apt_t,psu,sex,age,...,N_VA_RAE,N_CAROT,N_RETIN,N_B1,N_B2,N_NIAC,N_VITC,LF_secur_y,LF_BUYER,LF_SAFE
0,2018.02.01.,A651172801,A6511728,2016.0,1.0,1.0,2.0,A651,1.0,61.0,...,68.125541,817.506495,0.000000,1.812120,1.599025,17.241284,417.773500,1.0,1.0,1.0
1,2018.02.01.,A651172802,A6511728,2016.0,1.0,1.0,2.0,A651,2.0,57.0,...,91.302698,1095.561039,0.005945,1.075103,0.756771,5.302599,31.845311,1.0,1.0,1.0
2,2018.02.01.,A651183001,A6511830,2016.0,1.0,1.0,2.0,A651,2.0,41.0,...,261.852637,2147.567940,82.888642,1.093273,0.939505,16.655787,29.051934,1.0,2.0,2.0
3,2018.02.01.,A651183002,A6511830,2016.0,1.0,1.0,2.0,A651,1.0,39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018.02.01.,A651183003,A6511830,2016.0,1.0,1.0,2.0,A651,2.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 데이터 전처리

In [4]:
df_data_cut = pd.DataFrame(df_data, columns=[
    df_data.HE_HP.name ,
    df_data.region.name ,
    df_data.sex.name ,
    df_data.age.name ,
    df_data.incm.name ,
    df_data.ho_incm.name ,
    df_data.edu.name ,
    df_data.occp.name ,
    df_data.cfam.name ,
    df_data.allownc.name ,
    df_data.house.name ,
    df_data.live_t.name ,
    df_data.ainc_1.name ,
    df_data.ainc.name ,
    df_data.marri_1.name ,
    df_data.marri_2.name ,
    df_data.tins.name ,
    df_data.npins.name ,
    df_data.D_1_1.name ,
    df_data.D_2_1.name ,
    df_data.D_2_wk.name ,
    df_data.DI1_dg.name ,
    df_data.DI1_pr.name ,
    df_data.DI1_pt.name ,
    df_data.DI1_2.name ,
    df_data.DI2_dg.name ,
    df_data.DI2_pr.name ,
    df_data.DI3_dg.name ,
    df_data.DI3_pr.name ,
    df_data.DI4_dg.name ,
    df_data.DI4_pr.name ,
    df_data.DI5_dg.name ,
    df_data.DI5_pr.name ,
    df_data.DI6_dg.name ,
    df_data.DI6_pr.name ,
    df_data.DM1_dg.name ,
    df_data.DM1_pr.name ,
    df_data.DM2_dg.name ,
    df_data.DM2_pr.name ,
    df_data.DM3_dg.name ,
    df_data.DM3_pr.name ,
    df_data.DM4_dg.name ,
    df_data.DM4_pr.name ,
    df_data.DJ2_dg.name ,
    df_data.DJ2_pr.name ,
    df_data.DJ4_dg.name ,
    df_data.DJ4_pr.name ,
    df_data.DE1_dg.name ,
    df_data.DE1_pr.name ,
    df_data.DE2_dg.name ,
    df_data.DE2_pr.name ,
    df_data.DC1_dg.name ,
    df_data.DC1_pr.name ,
    df_data.DC2_dg.name ,
    df_data.DC2_pr.name ,
    df_data.DC3_dg.name ,
    df_data.DC3_pr.name ,
    df_data.DC4_dg.name ,
    df_data.DC4_pr.name ,
    df_data.DC5_dg.name ,
    df_data.DC5_pr.name ,
    df_data.DC6_dg.name ,
    df_data.DC6_pr.name ,
    df_data.DC7_dg.name ,
    df_data.DC7_pr.name ,
    df_data.DC11_dg.name ,
    df_data.DC11_pr.name ,
    df_data.DF2_dg.name ,
    df_data.DF2_pr.name ,
    df_data.DL1_dg.name ,
    df_data.DL1_pr.name ,
    df_data.DJ8_dg.name ,
    df_data.DJ8_pr.name ,
    df_data.DJ6_dg.name ,
    df_data.DJ6_pr.name ,
    df_data.DH4_dg.name ,
    df_data.DH4_pr.name ,
    df_data.DH2_dg.name ,
    df_data.DH2_pr.name ,
    df_data.DH3_dg.name ,
    df_data.DH3_pr.name ,
    df_data.DH6_dg.name ,
    df_data.DH6_pr.name ,
    df_data.DN1_dg.name ,
    df_data.DN1_pr.name ,
    df_data.DK8_dg.name ,
    df_data.DK8_pr.name ,
    df_data.DK9_dg.name ,
    df_data.DK9_pr.name ,
    df_data.DK4_dg.name ,
    df_data.DK4_pr.name ,
    df_data.BH9_11.name ,
    df_data.BH1.name ,
    df_data.BH1_1.name ,
    df_data.BH1_2.name ,
    df_data.BH1_3.name ,
    df_data.BH1_8.name ,
    df_data.BH1_6.name ,
    df_data.BH2_61.name ,
    df_data.BH2_62.name ,
    df_data.BH2_63.name ,
    df_data.BH2_66.name ,
    df_data.BH2_67.name ,
    df_data.BH2_64.name ,
    df_data.educ.name ,
    df_data.graduat.name ,
    df_data.EC1_1.name ,
    df_data.EC1_2.name ,
    df_data.EC_occp.name ,
    df_data.EC_stt_1.name ,
    df_data.EC_stt_2.name ,
    df_data.EC_wh.name ,
    df_data.EC_wht_0.name ,
    df_data.EC_wht_23.name ,
    df_data.EC_wht_5.name ,
    df_data.EC_lgw_2.name ,
    df_data.EC_lgw_4.name ,
    df_data.EC_lgw_5.name ,
    df_data.EC_pedu_1.name ,
    df_data.EC_pedu_2.name
])
total_column_count = df_data_cut.columns.size
print(total_column_count)
print(df_data_cut.shape)
df_data_cut.head()

120
(8150, 120)


,HE_HP,region,sex,age,incm,ho_incm,edu,occp,cfam,allownc,...,EC_stt_2,EC_wh,EC_wht_0,EC_wht_23,EC_wht_5,EC_lgw_2,EC_lgw_4,EC_lgw_5,EC_pedu_1,EC_pedu_2
0,3.0,1.0,1.0,61.0,3.0,3.0,4.0,7.0,3.0,20.0,...,8.0,8.0,8.0,888.0,88.0,3.0,1.0,1.0,3.0,3.0
1,2.0,1.0,2.0,57.0,3.0,3.0,4.0,6.0,3.0,20.0,...,2.0,1.0,2.0,50.0,1.0,3.0,1.0,1.0,3.0,3.0
2,1.0,1.0,2.0,41.0,3.0,3.0,4.0,1.0,5.0,20.0,...,2.0,2.0,2.0,24.0,1.0,2.0,1.0,1.0,8.0,5.0
3,3.0,1.0,1.0,39.0,3.0,3.0,4.0,3.0,5.0,20.0,...,1.0,1.0,1.0,61.0,1.0,5.0,1.0,1.0,8.0,5.0
4,1.0,1.0,2.0,38.0,3.0,3.0,4.0,7.0,5.0,20.0,...,8.0,8.0,8.0,888.0,88.0,3.0,1.0,1.0,5.0,5.0


## 고혈압 유병여부 NaN값 대상 삭제

In [5]:
df_data_cut.dropna(subset=['HE_HP'], inplace=True)
df_data_cut.shape

(6105, 120)

## 모든 값이 NaN인 경우 삭제

In [6]:
df_data_cut.dropna(how='all', inplace=True)
df_data_cut.reindex()
print(df_data_cut.shape)
df_data_cut.head()

(6105, 120)


,HE_HP,region,sex,age,incm,ho_incm,edu,occp,cfam,allownc,...,EC_stt_2,EC_wh,EC_wht_0,EC_wht_23,EC_wht_5,EC_lgw_2,EC_lgw_4,EC_lgw_5,EC_pedu_1,EC_pedu_2
0,3.0,1.0,1.0,61.0,3.0,3.0,4.0,7.0,3.0,20.0,...,8.0,8.0,8.0,888.0,88.0,3.0,1.0,1.0,3.0,3.0
1,2.0,1.0,2.0,57.0,3.0,3.0,4.0,6.0,3.0,20.0,...,2.0,1.0,2.0,50.0,1.0,3.0,1.0,1.0,3.0,3.0
2,1.0,1.0,2.0,41.0,3.0,3.0,4.0,1.0,5.0,20.0,...,2.0,2.0,2.0,24.0,1.0,2.0,1.0,1.0,8.0,5.0
3,3.0,1.0,1.0,39.0,3.0,3.0,4.0,3.0,5.0,20.0,...,1.0,1.0,1.0,61.0,1.0,5.0,1.0,1.0,8.0,5.0
4,1.0,1.0,2.0,38.0,3.0,3.0,4.0,7.0,5.0,20.0,...,8.0,8.0,8.0,888.0,88.0,3.0,1.0,1.0,5.0,5.0


## 특정 value 값을 NaN 처리

In [7]:
def setNanValues(in_df, name, nan_values) :
    
    start_na_count = in_df[name].isna().sum()
    
    uniq_values = sorted(map(float,in_df[name].dropna().unique()))
    
    in_df[name] = in_df[name].map(lambda x: np.nan if x in nan_values else x)
        
    #print(sorted(map(float,in_df[name].dropna().unique())))
        
    end_na_count = in_df[name].isna().sum()
    
    print("%s NaN count %s -> %s" % (name,start_na_count,end_na_count))
    
    return in_df

nan_df_data = df_data_cut.copy(deep=True)
nan_df_data = setNanValues(nan_df_data, nan_df_data.marri_1.name, [9.0])
nan_df_data = setNanValues(nan_df_data, nan_df_data.marri_2.name, [88.0, 99.0])
nan_df_data = setNanValues(nan_df_data, nan_df_data.EC_pedu_1.name, [88.0, 99.0])
nan_df_data = setNanValues(nan_df_data, nan_df_data.EC_pedu_2.name, [88.0, 99.0])
# 'DI4_dg', 'DI4_pr', 'DM1_dg', 'DM1_pr'
nan_df_data = setNanValues(nan_df_data, nan_df_data.DI4_dg.name, [8.0])
nan_df_data = setNanValues(nan_df_data, nan_df_data.DI4_pr.name, [8.0])
nan_df_data = setNanValues(nan_df_data, nan_df_data.DM1_dg.name, [8.0])
nan_df_data = setNanValues(nan_df_data, nan_df_data.DM1_pr.name, [8.0])
print(nan_df_data.shape)
nan_df_data.columns

marri_1 NaN count 0 -> 0
marri_2 NaN count 0 -> 941
EC_pedu_1 NaN count 0 -> 1042
EC_pedu_2 NaN count 0 -> 890
DI4_dg NaN count 271 -> 271
DI4_pr NaN count 271 -> 5928
DM1_dg NaN count 272 -> 272
DM1_pr NaN count 272 -> 5348
(6105, 120)


Index(['HE_HP', 'region', 'sex', 'age', 'incm', 'ho_incm', 'edu', 'occp',
       'cfam', 'allownc',
       ...
       'EC_stt_2', 'EC_wh', 'EC_wht_0', 'EC_wht_23', 'EC_wht_5', 'EC_lgw_2',
       'EC_lgw_4', 'EC_lgw_5', 'EC_pedu_1', 'EC_pedu_2'],
      dtype='object', length=120)

## NaN 값 처리

In [8]:
'''
strategy = mean, median, most_frequent, constant
'''
def setNanImputer(in_df, name, strategy="mean") :
    
    tmp_df = pd.DataFrame(in_df[name], columns=[name])
    
    imp = SimpleImputer(missing_values=np.nan, strategy=strategy)
    in_df[name] = imp.fit_transform(tmp_df)
        
    print('%s null count %s, na count %s' % (name, in_df[name].isnull().sum(), in_df[name].isna().sum()))
    
    return in_df


imp_df = nan_df_data.copy(deep=True)

imputer_infos = [
      ['ainc', 'mean']
]

for name, strategy in imputer_infos :
    imp_df = setNanImputer(imp_df, name, strategy=strategy)

ainc null count 0, na count 0


## One-Hot Encoding

In [9]:
def addOneHotEncodingColumns(in_df, name) :
    
    uniq_values = sorted(map(float,in_df[name].dropna().unique()))
    
    dummie = pd.get_dummies(in_df[name], dummy_na=True)
    dummie.columns=['%s_ONE_%s' % (name,str(n)) for n in uniq_values] + ['%s_ONE_NaN' % name]
    
    one_hot_df = pd.concat([in_df,dummie],axis=1)    
    one_hot_df.drop(columns=[name], inplace=True)
    
    return one_hot_df

one_hot_df = imp_df.copy(deep=True)
print(one_hot_df.shape)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.region.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.incm.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.ho_incm.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.edu.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.occp.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.EC_pedu_1.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.EC_pedu_2.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.marri_1.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.marri_2.name)
# 'DI4_dg', 'DI4_pr', 'DM1_dg', 'DM1_pr'
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.DI4_dg.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.DI4_pr.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.DM1_dg.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.DM1_pr.name)
print(one_hot_df.shape)
print(one_hot_df.columns)
one_hot_df.tail()

(6105, 120)
(6105, 187)
Index(['HE_HP', 'sex', 'age', 'cfam', 'allownc', 'house', 'live_t', 'ainc_1',
       'ainc', 'tins',
       ...
       'DI4_dg_ONE_NaN', 'DI4_pr_ONE_0.0', 'DI4_pr_ONE_1.0', 'DI4_pr_ONE_NaN',
       'DM1_dg_ONE_0.0', 'DM1_dg_ONE_1.0', 'DM1_dg_ONE_NaN', 'DM1_pr_ONE_0.0',
       'DM1_pr_ONE_1.0', 'DM1_pr_ONE_NaN'],
      dtype='object', length=187)


,HE_HP,sex,age,cfam,allownc,house,live_t,ainc_1,ainc,tins,...,DI4_dg_ONE_NaN,DI4_pr_ONE_0.0,DI4_pr_ONE_1.0,DI4_pr_ONE_NaN,DM1_dg_ONE_0.0,DM1_dg_ONE_1.0,DM1_dg_ONE_NaN,DM1_pr_ONE_0.0,DM1_pr_ONE_1.0,DM1_pr_ONE_NaN
8145,3.0,1.0,75.0,3.0,10.0,2.0,2.0,150.0,150.0,20.0,...,0,0,0,1,1,0,0,0,0,1
8146,3.0,2.0,72.0,3.0,10.0,2.0,2.0,150.0,150.0,20.0,...,0,0,1,0,0,1,0,0,1,0
8147,1.0,2.0,54.0,4.0,20.0,3.0,2.0,12000.0,1000.0,20.0,...,0,0,0,1,1,0,0,0,0,1
8148,3.0,1.0,56.0,4.0,20.0,3.0,2.0,12000.0,1000.0,20.0,...,0,0,0,1,1,0,0,0,0,1
8149,1.0,2.0,28.0,4.0,20.0,3.0,2.0,12000.0,1000.0,20.0,...,0,0,0,1,1,0,0,0,0,1


In [10]:
one_hot_df[one_hot_df.sex.name] = one_hot_df[one_hot_df.sex.name].astype(int)

## 결측치가 하나 이상 있는 컬럼 확인

In [11]:
nan_names = one_hot_df.columns[one_hot_df.isna().any()]
nan_names

Index([], dtype='object')

In [12]:
for index, name in enumerate(nan_names) :
    value_count = pd.value_counts(one_hot_df[name], dropna=False)
    nan_count = value_count[np.nan]
    print("[ %3d ] %s NaN count %s" % (index,name,nan_count))
    value_count.sort_index().plot(kind='bar', figsize=(15,5))
    plt.show()

# Train, Test data split

In [13]:
train_df, test_df = train_test_split(one_hot_df, test_size=0.25)
print(train_df.shape)
print(test_df.shape)

Y = one_hot_df.HE_HP.name
X = one_hot_df.columns.drop(Y).tolist()

(4578, 187)
(1527, 187)


# Decision Tree Classifier

In [14]:
def runDTR(x_name_list, y_name, train_df, test_df, max_depth=None) :
    
    stime = time.time()

    x_train_df = train_df[x_name_list]
    y_train_df = train_df[y_name]
    
    x_test_df = test_df[x_name_list]
    y_test_df = test_df[y_name]

    model = DecisionTreeClassifier(
          criterion = 'gini'
        , max_depth=max_depth
    ).fit(x_train_df,y_train_df)
    
    pdf_name = "./pdf_decision_tree_classifier_gini/target-%s_feature_count-%s_max_depth-%s" % (y_name,x_name_list.__len__(),max_depth)

    dot_data = tree.export_graphviz(model,
                                    out_file=None, 
                                    feature_names=x_name_list, 
                                    # class_names=['정상', '고혈압전단계', '고혈압']
                                    class_names=['1','2','3']
                                   )

    graph = graphviz.Source(dot_data) 
    graph.render(pdf_name)
    #graph.render("%s+%s" % (x_name,y_name))

    modelPrediction = model.predict(x_test_df)
    accuracyRate = accuracy_score(
        y_true=y_test_df
    ,   y_pred=modelPrediction
    )

    etime = time.time()
    
    print(pdf_name,', 정확도(%s) 소요시간(%s)' % (accuracyRate,etime-stime))
    
    return accuracyRate

In [15]:
'''
maxdepth가 높으면 처리 속도가 늦음
maxdepth가 낮으면 처리 속도가 빠름
'''
for depth in range(one_hot_df.columns.size, 5, -1) :
    accuracy_rate = runDTR(X, Y, train_df, test_df, depth)

# Random Forest Classifier

In [21]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(
      n_estimators=100
    , max_depth=20
    , random_state=0
)
clf.fit(train_df[X], train_df[Y])

for imp in zip(X,clf.feature_importances_) :
    print(imp)
print(clf.predict(test_df[X]))

('sex', 0.018377216600313048)
('age', 0.06381129785896265)
('cfam', 0.01680909683189645)
('allownc', 0.002381224374624741)
('house', 0.010395588718394789)
('live_t', 0.011395668899812859)
('ainc_1', 0.03252013356581929)
('ainc', 0.033208074772467955)
('tins', 0.007219563591873383)
('npins', 0.008433867197114735)
('D_1_1', 0.012265267453357884)
('D_2_1', 0.004332239488475365)
('D_2_wk', 0.007106293488308912)
('DI1_dg', 0.06877944246890252)
('DI1_pr', 0.06569421845174517)
('DI1_pt', 0.07470402620433977)
('DI1_2', 0.07816401274727969)
('DI2_dg', 0.008298021873794864)
('DI2_pr', 0.008161811517136441)
('DI3_dg', 0.0009075198692517652)
('DI3_pr', 0.0008530300264011775)
('DI5_dg', 0.0004438435269102241)
('DI5_pr', 0.0005259883844075486)
('DI6_dg', 0.0006095884119755683)
('DI6_pr', 0.000631989880053381)
('DM2_dg', 0.002028622826626941)
('DM2_pr', 0.002299504820540824)
('DM3_dg', 0.0007609450350998727)
('DM3_pr', 0.0007935166690023274)
('DM4_dg', 0.002115766445145506)
('DM4_pr', 0.0026835026731